<a href="https://colab.research.google.com/github/mrsferret/Machine-Learning-ITNPBD6-/blob/main/TFComputerVision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Building an Image Classifier Convolutional Neural Network in Keras with Tensorflow

- Uses a small part of the Cifar10 dataset
- Assumes you have downloaded the file `data_batch_1` from cifar (or the University of Stirling module webpage)
- Uses Keras to build the CNN

You can learn more about the Cifar10 data set here<br> https://www.cs.toronto.edu/~kriz/cifar.html<br>
You do not need to do that to complete this workbook, however.

## Imports
The data from Cifar10 is wrapped up in a pickle file, so we need the `pickle` library to unpack it. We also need numpy to hold the data and matplotlib if we want to look at the images

In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import pickle

## The Data
For this example, we are just using a small number of the Cifar images. They come in batches, so we will just use batch 1. You can download all the data from the Cifar web page, but if you are using this as part of a course at the University of Stirling, the two files you need should have been made available on your learning platform. They are:

- data_batch_1
- batches.meta

If they are not in the same folder as this code on your computer, you will need to give their path in the unpickle calls below.

In [ ]:
def unpickle(file):
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

imgs = unpickle('data_batch_1')
meta_data_dict = unpickle("batches.meta")


## Now Let's Look at What We've Got
Here we extract the data and the labels

In [ ]:
print(imgs.keys())
print(meta_data_dict)
labels = meta_data_dict[b'label_names']
labels = [l.decode("utf-8") for l in labels]
print(labels)
batch1 = imgs[b'data']

## Reshape the Arrays
The data are in 1D arrays, so we reshape them to 32 x 32 arrays of pixel values

In [ ]:
batch1 = batch1.reshape((len(batch1), 3, 32, 32))
batch1 = np.rollaxis(batch1, 1, 4)

## Plot an Image
We can see what one of the images looks like by plotting it. Change the index `[0]` to another number to see different examples, or plot a few in a loop.

In [ ]:
plt.imshow(batch1[0])
plt.show()
label_ix = imgs[b'labels'][0]
print(labels[label_ix])

## Finally, Tensorflow
No we import Tensorflow and the parts of Keras that we will need. We also need the label encoder from scikit-learn to transform the labels to integers.

In [ ]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras import layers
from sklearn.preprocessing import LabelEncoder

# Limit memory usage for GPU
# Leave commented unless you know what you are doing and have TFGPU installed
#import os
#os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
#print(tf.__version__)

x = batch1.astype('float32')
y = imgs[b'labels']
y = LabelEncoder().fit_transform(y)
print(y)

## Build the model
This is where you can experiment with different layers. You can see the basic pattern: Convolution, Pooling (repeat a few times), Dense, then output. We use softmax at the output to force the probabilities across the classes to sum to one.

`SparseCategoricalCrossentropy` loss converts the integer label index values in `y` to one-hot encoded targets and applies the Cross Entropy cost function.

Try different numbers of epochs (training cycles) to see how long the network needs to train before it stops improving.

Try changing the number of filters at each convolution level. Can you get away with fewer than 64 at the last level? What if you use more in the first one instead?

You can also try different activation functions ('sigmoid' for example) and add or remove layers to see how that effects the quality of the learning.

In [ ]:
# Build model

# Begin the sequence of the model building
model = Sequential()

# First layer takes an input of 32 x 32 pixels by 3 colours with input_shape=(32, 32, 3)
model.add(layers.Conv2D(5, (3, 3), activation='relu', input_shape=(32, 32, 3)))
# It has 5 three by 3 filters and uses a relu activation

model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten()) # Flatten all the convolution layers into a single layer
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))
# fit the model
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

history = model.fit(x, y, epochs=10)

## Plot the Accuracy by Epoch

In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')

## Make a Classification
Change `test_img_ix` to choose different images from the training data.

In [ ]:
test_img_ix = 20
plt.imshow(batch1[test_img_ix])
plt.show()
label_ix = imgs[b'labels'][test_img_ix]
print("Label = ", labels[label_ix])
class_scores = model.predict(x[test_img_ix:test_img_ix+1])
best = np.argmax(class_scores)
print("Classification =", labels[best])

## Plot the Probabilities
Plot the probability of the image belonging to each possible class

In [ ]:
plt.bar(range(10), class_scores[0])
plt.xticks(range(10), labels)
plt.show()

## Advanced Stuff Below!
Finally, we examine the model to understand its structure. Use `model.summary()` to see the overall structure.

In [ ]:
model.summary()